<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0502Minnichi_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# みんなの日本語 言語モデル の作成
- date: 2021_0502
- author: 浅川伸一
- 概要:

岩下先生からいただいた みんなの日本語 テキストデータ から言語モデルを作成する

In [1]:
import numpy as np

# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

import matplotlib.pyplot as plt

In [ ]:
# minnchi.txt を upload してださい。
from google.colab import files
files.upload()

In [ ]:
print('# データの読み込み all に格納')
all = open('minnichi.txt', 'r').read().strip().split('\n')

print('#all 内の "。" の位置を探して，文を分割。結果を data に格納')
text_data = []
for line in all:
    positions = []
    for i, word in enumerate(line):
        if word == '。':
            positions.append(i)
    c = []
    p0 = 0
    for p in positions:
        text_data.append(line[p0:p+1])
        p0 = p+1
    if len(positions) == 0:
        text_data.append(line)

print(f'#総文数: {len(text_data)}')

In [ ]:
print('#data 内の単語頻度を計測し word_freq に格納')
word_freq = {}
for line in text_data:
    for word in line.split():
        if not word in word_freq:
            word_freq[word] = 1
        else:
            word_freq[word] += 1

print('#後の処理のため 単語リスト word_list を作成')
word_list = sorted(list(word_freq))
n_vocab = len(word_list)
print(f'#総語彙数: {n_vocab}')

print('#ワンホットベクトルを作成するため wrd2idx, idx2wrd を作成')
wrd2idx = {w:i for i, w in enumerate(word_list)}
idx2wrd = {i:w for i, w in enumerate(word_list)}

In [ ]:
print('#text_data を単語 ID へ変換した X を作成')
X = []
for line in text_data:
    words = line.split()
    X.append([wrd2idx[word] for word in words])


In [ ]:
print(len(X))
print('#データチェック')
No = int(input('チェックのため数字を入力してください'))
print(f'X[{No}]:\n単語 ID 系列:  {X[No]} \nIDを単語に変換:{[idx2wrd[id] for id in X[No]]}\n元データ:      {text_data[No]}')

In [ ]:
print('#単語ID からなるデータ X を作成')
X = []
for line in text_data:
    words = line.split()
    X.append([wrd2idx[word] for word in words])

print('#最大文長となるデータを探して表示する')
max_len, line_no = 0, 0
for i, line in enumerate(X):
    if len(line) > max_len:
        max_len = len(line)
        line_no = i
        #print(line_no, line)
        print(line_no, "".join(idx2wrd[id] for id in line))

print(f'最大文長: {max_len} 単語，データ番号: {line_no}')

In [ ]:
print('#ソフトマックス関数の定義')
def softmax(x, beta=1):
    xt = np.exp(beta * x - np.mean(beta * x))
    return xt / np.sum(xt)

In [ ]:
print('#ハイパーパラメータ の設定: 学習率 lr, 中間層ニューロン数 n_hid, 最大系列長 seq_len')
lr = 1e-1          #学習率
n_hid = 20         #中間層のニューロン数
seq_len = max_len  #RNN の時間ステップ数

n_data = len(X)
n_vocab = len(word_list)

print('#推定すべきパラメータ: 結合係数行列とバイアス項の初期化')
Wxh = np.random.randn(n_hid, n_vocab) * 0.01   # 入力層 -> 中間層
Whh = np.random.randn(n_hid, n_hid)   * 0.01   # 中間層 -> 中間層 リカレント結合
Why = np.random.randn(n_vocab, n_hid) * 0.01   # 中間層 -> 出力層
bh = np.zeros((n_hid, 1))                      # 中間層バイアス項
by = np.zeros((n_vocab, 1))                    # 出力層バイアス項
print(f'# Wxh:{Wxh.shape}, Whh:{Whh.shape}, Why:{Why.shape}, bh:{bh.shape}, by:{by.shape}')

In [ ]:
print('#前向きパス(次単語予測) と 後向きパス(誤差逆伝播) 関数の定義')
def rnn_forward(inputs, targets, h_prev, Wxh=Wxh, Whh=Whh, Why=Why, bh=bh, by=by):
    """RNN 前向きパス 

    引数:
    - inputs: int のリスト 
    - targets: int のリスト
    - h_prev: 隠れ層の初期状態 Hx1 (H 行 x 1 列)
    
    戻り値:
    - loss: np.array
        損失値，出力層のニューロンごとの損失値
    - prob: np.array
        各時刻ごとの各項目(単語)の予測確率 足し合わせると 1 になる
    - H: dict
        各時刻ごとの中間層の状態
    - X: dict
        各時刻ごとの入力層の状態

    
    y0       y1         y2
    |         |         | Why
    h --Whh-- h --Whh-- h       h と y にはバイアス項 bh, hy 付く
    |         |         | Wxh
    x0       x1         x2
    """
    X, H, Y, prob = {}, {}, {}, {}
    H[-1] = np.copy(h_prev)
    loss = 0
    
    for t in range(len(inputs)):
        
        X[t] = np.zeros((n_vocab,1))           # ワンホット表現
        X[t][inputs[t]] = 1
        
        #隠れ層の状態
        H[t] = np.tanh(np.dot(Wxh, X[t]) + np.dot(Whh, H[t-1]) + bh)
        
        Y[t] = np.dot(Why, H[t]) + by           #次項目(モーラ) の対数確率
        prob[t] = softmax(Y[t])                 #次項目の確率
        loss += -np.log(prob[t][targets[t],0])  #交差エントロピー損失
        
    return loss, prob, H, X


def rnn_backword(inputs, targets, prob_, h_, x_, Wxh=Wxh, Whh=Whh, Why=Why, bh=bh, by=by):
    """RNN の後向きパス: 勾配計算
    戻り値: 6 つ
    - dWxh, dWhh, dWhy, dbh, dby: 勾配 
    - h_[len(inputs)-1]: 隠れ層の状態
        
    y0       y1         y2
    |         |         | Why
    h --Whh-- h --Whh-- h       h と y にはバイアス項 bh, hy 付く
    |         |         | Wxh
    x0       x1         x2
    """
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dh_next = np.zeros_like(h_[0])
    
    for t in reversed(range(len(inputs))):
        #y への逆伝播
        #参考 http://cs231n.github.io/neural-networks-case-study/#grad
        dy = np.copy(prob_[t])
        dy[targets[t]] -= 1               
        dWhy += np.dot(dy, h_[t].T)
        dby += dy
        
        dh = np.dot(Why.T, dy) + dh_next  #中間層へ誤差逆伝播
        delta = (1 - h_[t] * h_[t]) * dh  #tanh での誤差逆伝播

        dbh  += delta
        dWxh += np.dot(delta, x_[t].T)
        dWhh += np.dot(delta, h_[t-1].T)
        dh_next = np.dot(Whh.T, delta)

    #勾配爆発緩和のためのクリッピング
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)       

    return dWxh, dWhh, dWhy, dbh, dby, h_[len(inputs)-1]

In [31]:
def sample_seq(h_prev, seed, n):
    """モデルを用いて項目(文字)番号の系列をランダムサンプリングを繰り返し，
    n 個からなる項目番号の系列を返す
    
    引数:
    - h_prev: np.array
        前時刻の中間層の状態 中間層の素子数 x 1の行列
    - seed: int
        文字id 時刻 0 で与える文字ID 番号
    
    戻り値:
    - idxes: list()
        項目番号列からなる予測系列のリスト
    """
    x = np.zeros((n_vocab, 1))
    hid = h_prev
    x[seed] = 1
    idxes = []
    for t in range(n):
        hid = np.tanh(np.dot(Wxh, x) + np.dot(Whh, hid) + bh)
        out = np.dot(Why, hid) + by
        prob = softmax(out)
        idx = np.random.choice(range(n_vocab), p=prob.ravel())
        #if idx2wrd[idx] == '。':
        #    return idxes
        x = np.zeros((n_vocab, 1))
        x[idx] = 1
    return idxes


def gen_seq(h_prev, seed, n):
    """モデルを用いて項目(文字)番号の系列を生成して，
    n 個からなる項目番号の系列を返す
    sample_seq との相違は，
    
    引数:
    - h_prev: np.array
        前時刻の中間層の状態 中間層の素子数 x 1の行列
    - seed: int
        文字id 時刻 0 で与える文字番号
    
    戻り値:
    - idxes: list()
        項目番号列からなる予測系列のリスト
    """
    x = np.zeros((n_vocab, 1))
    x[seed] = 1
    hid = h_prev
    idxes = []
    for t in range(n):
        hid = np.tanh(np.dot(Wxh, x) + np.dot(Whh, hid) + bh)
        out = np.dot(Why, hid) + by
        prob = softmax(out)
        idx = np.argmax(prob)
        idxes.append(idx)
        x = np.zeros((n_vocab, 1))
        x[idx] = 1
    return idxes


In [ ]:
print('#動作確認 N 個の文をランダムサンプリングして前向きパスに通してエラーが発生しないかチェックする')
N = int(input('数字を入力してください:'))
if N == 0 or N > len(X): N=3
for line_id in np.random.permutation(len(X))[:N]:
    x = X[line_id][:-1]           #入力系列データ 各時刻における入力単語系列
    y = X[line_id][1:]            #出力系列データ 次時刻における教師単語系列
    h_prev = np.zeros((n_hid,1))  #中間層状態を 0 でクリア
    loss, prob, _, _ = rnn_forward(x, y, h_prev)  #  前向き処理の実行
    print(f'line no.:{line_id} 損失値={loss:.3f}, 正解系列(y):{y} ->「{"".join(idx2wrd[idx] for idx in y)}」text_data:「{text_data[line_id]}」')


In [ ]:
# m で始まる変数は，Adagrad で用いるメモリ変数。それぞれ，
# mWxh: 入力から中間層への結合係数行列
# mWhh: 中間層へのリカレント結合係数行列
# mWhy: 中間層から出力層への結合係数行列
# mbh: 中間層のバイアス項
# mby: 出力層のバイアス項
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 

smooth_loss = -np.log(1.0/n_vocab) * max_len      #損失値の理論的上限値

max_iter = 10 ** 2
interval = max_iter >> 2
losses = []

for iter in range(max_iter):
    
    idxs = np.random.permutation(len(X))  # 全データをシャッフル
    idxs = range(len(X))
    for idx in idxs:
        
        if len(X[idx]) == 1:
            continue
        inputs = X[idx][:-1]
        targets = X[idx][1:]

        #前向き処理
        h_prev = np.zeros((n_hid,1)) # reset RNN memory
        loss, prob, h_, x_ = rnn_forward(inputs, targets, h_prev)
        
        #損失値の処理
        losses.append(loss)
        smooth_loss = smooth_loss * 0.999 + loss * 0.001

        #後向き処理
        dWxh, dWhh, dWhy, dbh, dby, h_ = rnn_backword(inputs, targets, prob, h_, x_)
        
        #Adagrad によるパラメータ更新
        for param, _delta, _Hessian in zip([Wxh, Whh, Why, bh, by], 
                                           [dWxh, dWhh, dWhy, dbh, dby], 
                                           [mWxh, mWhh, mWhy, mbh, mby]):
            _Hessian += _delta * _delta
        param += -lr * _delta / np.sqrt(_Hessian + 1e-8) # 更新

    #途中結果の印字
    if iter % interval == 0:
        pred_idxes = gen_seq(h_prev, inputs, 50)
        pred_text = ''.join(idx2wrd[idx] for idx in pred_idxes)
        print(f'--- 訓練回数:{iter:<5d} ---\n{pred_text}\n---')
        print(f'損失値:{smooth_loss:.3f}') # print progress


h_prev = np.zeros((n_hid,1)) # reset RNN memory
pred_idxes = gen_seq(h_prev, X[0][1:], 50)
pred_text = ''.join(idx2wrd[idx] for idx in pred_idxes)
print(f'----\n{pred_text}\n----')
plt.plot(losses)


--- 訓練回数:0     ---
。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。
---
損失値:205.545
--- 訓練回数:25    ---
。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。
---
損失値:57.620
